In [109]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 


import html
import os

In [110]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [112]:
#open a bibtex file
parser = bibtex.Parser()
bibdata = parser.parse_file("pubs.bib")


import re
#loop through the individual references
for bib_id in bibdata.entries:
    b = bibdata.entries[bib_id].fields
    print("%#"*40)
    print(b)
    try:
        # change these lines to create a SQL insert
 #       print(f'{b["year"]}-{b["month"]}\t{b["title"].replace("{", "").replace("}","")}')
 #       print(b["journal"])
        pub_date = f'{b["year"]}'
        
        for x in ["month","day"]:
            if x in b.keys(): pub_date = pub_date+"-"+b[x]
        url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", f'{b["title"].replace("{", "").replace("}","").replace(" ","-")}')
        #deal with multiple authors
       # for author in bibdata.entries[bib_id].persons["author"]:
       #     print(author.first_names, author.last_names)
        
        
        md_filename = str(pub_date) + "-" + url_slug + ".md"
        html_filename = str(pub_date) + "-" + url_slug
        year = b["year"]

        ## YAML variables

        md = "---\ntitle: \""   + b["title"].replace("{", "").replace("}","") + '"\n'

        md += """collection: publications"""

        md += """\npermalink: /publication/""" + html_filename

        if "note" in b.keys():
            if len(str(b["note"])) > 5:
                md += "\nexcerpt: '" + html_escape(b["note"]) + "'"

        md += "\ndate: " + str(pub_date) 

        md += "\nvenue: '" + html_escape(b["journal"]) + "'"

        if "url" in b.keys():
            if len(str(b["url"])) > 5:
                md += "\npaperurl: '" + b["url"] + "'"

        #md += "\ncitation: '" + html_escape(b) + "'"

        md += "\n---"

        ## Markdown description for individual page
        if "note" in b.keys():
            if len(str(b["note"])) > 5:
                md += "\n" + html_escape(b["note"]) + "\n"

        if "url" in b.keys():
            if len(str( b["url"])) > 5:
                md += "\n[Download paper here](" + b["url"] + ")\n" 
        md += "\nRecommended citation - Use [Google Scholar](https://scholar.google.com/scholar?q="+html.escape(b["title"].replace(" ","+"))+") for appropriate Citation "

        md_filename = os.path.basename(md_filename)

        with open("../_publications/" + md_filename, 'w') as f:
            f.write(md)
        
        print(md_filename)
        print(md)
    
    # field may not exist for a reference
    except(KeyError):
        print(KeyError.args)                  
        continue
    print("%#"*40)


%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#
OrderedCaseInsensitiveDict([('title', '{[Invited, Under Review] Autonomous Decentralized Lightweight Swarms}'), ('Author+an', '3=highlight'), ('journal', 'IEEE Consumer Electronics Magazine'), ('year', '2019'), ('publisher', 'IEEE')])
2019--Autonomous-Decentralized-Lightweight-Swarms.md
---
title: "[Invited, Under Review] Autonomous Decentralized Lightweight Swarms"
collection: publications
permalink: /publication/2019--Autonomous-Decentralized-Lightweight-Swarms
date: 2019
venue: 'IEEE Consumer Electronics Magazine'
---
Recommended citation - Use [Google Scholar](https://scholar.google.com/scholar?q={[Invited,+Under+Review]+Autonomous+Decentralized+Lightweight+Swarms}) for appropriate Citation 
%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#
%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#
OrderedCaseInsensitiveDict([('ISSN', '1751-8601'), (

PluginNotFound: plugin pybtex.database.output.plain not found